In [1]:
!pip install dynet

Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
    100% |████████████████████████████████| 27.8MB 876kB/s 
    100% |████████████████████████████████| 3.4MB 10.2MB/s 


In [2]:
!git clone https://github.com/neubig/nn4nlp-code.git

fatal: destination path 'nn4nlp-code' already exists and is not an empty directory.


In [0]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

In [0]:
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]

In [0]:
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

In [0]:
train = list(read_dataset("nn4nlp-code/data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("nn4nlp-code/data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

In [0]:
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [0]:
W_sm = model.add_lookup_parameters((nwords, ntags))
b_sm = model.add_parameters((ntags))

In [0]:
def calc_scores(words):
  dy.renew_cg()
  b_sm_exp = dy.parameter(b_sm)
  score = dy.esum([dy.lookup(W_sm, x) for x in words])
  return score + b_sm_exp

In [11]:
for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform testing
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=0.3377, time=0.29s
iter 0: test acc=0.3887
iter 1: train loss/sent=0.3352, time=0.28s
iter 1: test acc=0.3891
iter 2: train loss/sent=0.3333, time=0.27s
iter 2: test acc=0.3900
iter 3: train loss/sent=0.3310, time=0.27s
iter 3: test acc=0.3882
iter 4: train loss/sent=0.3288, time=0.27s
iter 4: test acc=0.3851
iter 5: train loss/sent=0.3269, time=0.28s
iter 5: test acc=0.3887
iter 6: train loss/sent=0.3247, time=0.29s
iter 6: test acc=0.3887
iter 7: train loss/sent=0.3227, time=0.29s
iter 7: test acc=0.3910
iter 8: train loss/sent=0.3205, time=0.28s
iter 8: test acc=0.3914
iter 9: train loss/sent=0.3186, time=0.29s
iter 9: test acc=0.3878
iter 10: train loss/sent=0.3165, time=0.29s
iter 10: test acc=0.3905
iter 11: train loss/sent=0.3144, time=0.28s
iter 11: test acc=0.3878
iter 12: train loss/sent=0.3126, time=0.28s
iter 12: test acc=0.3814
iter 13: train loss/sent=0.3110, time=0.29s
iter 13: test acc=0.3869
iter 14: train loss/sent=0.3091, time=0.29s
iter 14: t

iter 53: train loss/sent=0.2509, time=0.30s
iter 53: test acc=0.3928
iter 54: train loss/sent=0.2498, time=0.29s
iter 54: test acc=0.3941
iter 55: train loss/sent=0.2485, time=0.28s
iter 55: test acc=0.3864
iter 56: train loss/sent=0.2474, time=0.29s
iter 56: test acc=0.3919
iter 57: train loss/sent=0.2463, time=0.29s
iter 57: test acc=0.3946
iter 58: train loss/sent=0.2452, time=0.28s
iter 58: test acc=0.3932
iter 59: train loss/sent=0.2438, time=0.30s
iter 59: test acc=0.3882
iter 60: train loss/sent=0.2428, time=0.30s
iter 60: test acc=0.3919
iter 61: train loss/sent=0.2416, time=0.29s
iter 61: test acc=0.3928
iter 62: train loss/sent=0.2405, time=0.29s
iter 62: test acc=0.3910
iter 63: train loss/sent=0.2395, time=0.29s
iter 63: test acc=0.3955
iter 64: train loss/sent=0.2384, time=0.28s
iter 64: test acc=0.3928
iter 65: train loss/sent=0.2373, time=0.28s
iter 65: test acc=0.3882
iter 66: train loss/sent=0.2361, time=0.29s
iter 66: test acc=0.3873
iter 67: train loss/sent=0.2353, t